In [1]:
import pandas as pd

df = pd.read_csv("TabularDatasets/list_attr_celeba.csv", header='infer')
df

,image_id,5_o_Clock_Shadow,Arched_Eyebrows,Attractive,Bags_Under_Eyes,Bald,Bangs,Big_Lips,Big_Nose,Black_Hair,...,Sideburns,Smiling,Straight_Hair,Wavy_Hair,Wearing_Earrings,Wearing_Hat,Wearing_Lipstick,Wearing_Necklace,Wearing_Necktie,Young
0,000001.jpg,-1,1,1,-1,-1,-1,-1,-1,-1,...,-1,1,1,-1,1,-1,1,-1,-1,1
1,000002.jpg,-1,-1,-1,1,-1,-1,-1,1,-1,...,-1,1,-1,-1,-1,-1,-1,-1,-1,1
2,000003.jpg,-1,-1,-1,-1,-1,-1,1,-1,-1,...,-1,-1,-1,1,-1,-1,-1,-1,-1,1
3,000004.jpg,-1,-1,1,-1,-1,-1,-1,-1,-1,...,-1,-1,1,-1,1,-1,1,1,-1,1
4,000005.jpg,-1,1,1,-1,-1,-1,1,-1,-1,...,-1,-1,-1,-1,-1,-1,1,-1,-1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202594,202595.jpg,-1,-1,1,-1,-1,-1,1,-1,-1,...,-1,-1,-1,-1,-1,-1,1,-1,-1,1
202595,202596.jpg,-1,-1,-1,-1,-1,1,1,-1,-1,...,-1,1,1,-1,-1,-1,-1,-1,-1,1
202596,202597.jpg,-1,-1,-1,-1,-1,-1,-1,-1,1,...,-1,1,-1,-1,-1,-1,-1,-1,-1,1
202597,202598.jpg,-1,1,1,-1,-1,-1,1,-1,1,...,-1,1,-1,1,1,-1,1,-1,-1,1


In [2]:
df.columns

Index(['image_id', '5_o_Clock_Shadow', 'Arched_Eyebrows', 'Attractive',
       'Bags_Under_Eyes', 'Bald', 'Bangs', 'Big_Lips', 'Big_Nose',
       'Black_Hair', 'Blond_Hair', 'Blurry', 'Brown_Hair', 'Bushy_Eyebrows',
       'Chubby', 'Double_Chin', 'Eyeglasses', 'Goatee', 'Gray_Hair',
       'Heavy_Makeup', 'High_Cheekbones', 'Male', 'Mouth_Slightly_Open',
       'Mustache', 'Narrow_Eyes', 'No_Beard', 'Oval_Face', 'Pale_Skin',
       'Pointy_Nose', 'Receding_Hairline', 'Rosy_Cheeks', 'Sideburns',
       'Smiling', 'Straight_Hair', 'Wavy_Hair', 'Wearing_Earrings',
       'Wearing_Hat', 'Wearing_Lipstick', 'Wearing_Necklace',
       'Wearing_Necktie', 'Young'],
      dtype='object')

In [3]:
hair_cols = ['image_id','Black_Hair', 'Blond_Hair', 'Brown_Hair', 'Gray_Hair']
df_hair = df[hair_cols]
df_hair.head()

,image_id,Black_Hair,Blond_Hair,Brown_Hair,Gray_Hair
0,000001.jpg,-1,-1,1,-1
1,000002.jpg,-1,-1,1,-1
2,000003.jpg,-1,-1,-1,-1
3,000004.jpg,-1,-1,-1,-1
4,000005.jpg,-1,-1,-1,-1


Test with 100 samples:

In [4]:
import cv2
import pandas as pd
import numpy as np
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from multiprocessing import Pool
import os


# Function to load and preprocess a single image
def process_image(image_name):
    image_path = os.path.join(image_folder_path, image_name)
    image = cv2.imread(image_path)
    if image is None:
        print(f"Unable to open {image_path}. Skipping.")
        return None
    image = cv2.resize(image, (32, 32))  # use a lower resolution
    return image.flatten()

# Load dataframe
#df = pd.read_csv('your_dataframe.csv')

# Randomly sample 10,000 images from the dataframe
df_sample = df_hair.sample(n=100, random_state=1)

# Load and preprocess images
image_folder_path = 'img_align_celeba/'

# Use multiprocessing to load and preprocess images
with Pool(os.cpu_count()) as p:
    images = p.map(process_image, df_sample['image_id'])

# Filter out any images that couldn't be opened
images = [image for image in images if image is not None]

# Convert list of images to numpy array
images = np.array(images)

# Prepare labels
labels = df_sample[['Black_Hair', 'Blond_Hair', 'Brown_Hair', 'Gray_Hair']].values
labels = np.argmax(labels, axis=1)  # convert from one-hot encoding to single label

# Split dataset into training set and test set
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

# Scale data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Train SVM
clf = svm.SVC(gamma='scale')
clf.fit(X_train, y_train)

# Check accuracy on test set
accuracy = clf.score(X_test, y_test)
print('Accuracy: ', accuracy)


Accuracy:  0.55


Test with 1000 samples:

In [6]:
import cv2
import pandas as pd
import numpy as np
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from multiprocessing import Pool
import os


# Function to load and preprocess a single image
def process_image(image_name):
    image_path = os.path.join(image_folder_path, image_name)
    image = cv2.imread(image_path)
    if image is None:
        print(f"Unable to open {image_path}. Skipping.")
        return None
    image = cv2.resize(image, (32, 32))  # use a lower resolution
    return image.flatten()

# Load dataframe
#df = pd.read_csv('your_dataframe.csv')

# Randomly sample 10,000 images from the dataframe
df_sample = df_hair.sample(n=1000, random_state=1)

# Load and preprocess images
image_folder_path = 'img_align_celeba/'

# Use multiprocessing to load and preprocess images
with Pool(os.cpu_count()) as p:
    images = p.map(process_image, df_sample['image_id'])

# Filter out any images that couldn't be opened
images = [image for image in images if image is not None]

# Convert list of images to numpy array
images = np.array(images)

# Prepare labels
labels = df_sample[['Black_Hair', 'Blond_Hair', 'Brown_Hair', 'Gray_Hair']].values
labels = np.argmax(labels, axis=1)  # convert from one-hot encoding to single label

# Split dataset into training set and test set
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

# Scale data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Train SVM
clf = svm.SVC(gamma='scale')
clf.fit(X_train, y_train)

# Check accuracy on test set
accuracy = clf.score(X_test, y_test)
print('Accuracy: ', accuracy)


Accuracy:  0.635


Same test but with higher resolution images:

In [8]:
import cv2
import pandas as pd
import numpy as np
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from multiprocessing import Pool
import os


# Function to load and preprocess a single image
def process_image(image_name):
    image_path = os.path.join(image_folder_path, image_name)
    image = cv2.imread(image_path)
    if image is None:
        print(f"Unable to open {image_path}. Skipping.")
        return None
    image = cv2.resize(image, (64, 64))  # use a lower resolution
    return image.flatten()

# Load dataframe
#df = pd.read_csv('your_dataframe.csv')

# Randomly sample 10,000 images from the dataframe
df_sample = df_hair.sample(n=1000, random_state=1)

# Load and preprocess images
image_folder_path = 'img_align_celeba/'

# Use multiprocessing to load and preprocess images
with Pool(os.cpu_count()) as p:
    images = p.map(process_image, df_sample['image_id'])

# Filter out any images that couldn't be opened
images = [image for image in images if image is not None]

# Convert list of images to numpy array
images = np.array(images)

# Prepare labels
labels = df_sample[['Black_Hair', 'Blond_Hair', 'Brown_Hair', 'Gray_Hair']].values
labels = np.argmax(labels, axis=1)  # convert from one-hot encoding to single label

# Split dataset into training set and test set
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

# Scale data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Train SVM
clf = svm.SVC(gamma='scale')
clf.fit(X_train, y_train)

# Check accuracy on test set
accuracy = clf.score(X_test, y_test)
print('Accuracy: ', accuracy)


Accuracy:  0.64


Same test but with lower resolution images:

In [11]:
import cv2
import pandas as pd
import numpy as np
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from multiprocessing import Pool
import os


# Function to load and preprocess a single image
def process_image(image_name):
    image_path = os.path.join(image_folder_path, image_name)
    image = cv2.imread(image_path)
    if image is None:
        print(f"Unable to open {image_path}. Skipping.")
        return None
    image = cv2.resize(image, (16, 16))  # use a lower resolution
    return image.flatten()

# Load dataframe
#df = pd.read_csv('your_dataframe.csv')

# Randomly sample 10,000 images from the dataframe
df_sample = df_hair.sample(n=1000, random_state=1)

# Load and preprocess images
image_folder_path = 'img_align_celeba/'

# Use multiprocessing to load and preprocess images
with Pool(os.cpu_count()) as p:
    images = p.map(process_image, df_sample['image_id'])

# Filter out any images that couldn't be opened
images = [image for image in images if image is not None]

# Convert list of images to numpy array
images = np.array(images)

# Prepare labels
labels = df_sample[['Black_Hair', 'Blond_Hair', 'Brown_Hair', 'Gray_Hair']].values
labels = np.argmax(labels, axis=1)  # convert from one-hot encoding to single label

# Split dataset into training set and test set
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

# Scale data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Train SVM
clf = svm.SVC(gamma='scale')
clf.fit(X_train, y_train)

# Check accuracy on test set
accuracy = clf.score(X_test, y_test)
print('Accuracy: ', accuracy)


Accuracy:  0.62


Test with 10,000 samples:

In [31]:
import cv2
import pandas as pd
import numpy as np
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from multiprocessing import Pool
import os


# Function to load and preprocess a single image
def process_image(image_name):
    image_path = os.path.join(image_folder_path, image_name)
    image = cv2.imread(image_path)
    if image is None:
        print(f"Unable to open {image_path}. Skipping.")
        return None
    image = cv2.resize(image, (32, 32))  # use a lower resolution
    return image.flatten()

# Load dataframe
#df = pd.read_csv('your_dataframe.csv')

# Randomly sample 10,000 images from the dataframe
df_sample = df_hair.sample(n=10000, random_state=1)

# Load and preprocess images
image_folder_path = 'img_align_celeba/'

# Use multiprocessing to load and preprocess images
with Pool(os.cpu_count()) as p:
    images = p.map(process_image, df_sample['image_id'])

# Filter out any images that couldn't be opened
images = [image for image in images if image is not None]

# Convert list of images to numpy array
images = np.array(images)

# Prepare labels
labels = df_sample[['Black_Hair', 'Blond_Hair', 'Brown_Hair', 'Gray_Hair']].values
labels = np.argmax(labels, axis=1)  # convert from one-hot encoding to single label

# Split dataset into training set and test set
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

# Scale data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Train SVM
clf = svm.SVC(gamma='scale')
clf.fit(X_train, y_train)

# Check accuracy on test set
accuracy = clf.score(X_test, y_test)
print('Accuracy: ', accuracy)


Accuracy:  0.73


Same test with lower resolution:

In [12]:
import cv2
import pandas as pd
import numpy as np
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from multiprocessing import Pool
import os


# Function to load and preprocess a single image
def process_image(image_name):
    image_path = os.path.join(image_folder_path, image_name)
    image = cv2.imread(image_path)
    if image is None:
        print(f"Unable to open {image_path}. Skipping.")
        return None
    image = cv2.resize(image, (16, 16))  # use a lower resolution
    return image.flatten()

# Load dataframe
#df = pd.read_csv('your_dataframe.csv')

# Randomly sample 10,000 images from the dataframe
df_sample = df_hair.sample(n=10000, random_state=1)

# Load and preprocess images
image_folder_path = 'img_align_celeba/'

# Use multiprocessing to load and preprocess images
with Pool(os.cpu_count()) as p:
    images = p.map(process_image, df_sample['image_id'])

# Filter out any images that couldn't be opened
images = [image for image in images if image is not None]

# Convert list of images to numpy array
images = np.array(images)

# Prepare labels
labels = df_sample[['Black_Hair', 'Blond_Hair', 'Brown_Hair', 'Gray_Hair']].values
labels = np.argmax(labels, axis=1)  # convert from one-hot encoding to single label

# Split dataset into training set and test set
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

# Scale data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Train SVM
clf = svm.SVC(gamma='scale')
clf.fit(X_train, y_train)

# Check accuracy on test set
accuracy = clf.score(X_test, y_test)
print('Accuracy: ', accuracy)


Accuracy:  0.7275


Test with 25,000 samples: 

In [15]:
import cv2
import pandas as pd
import numpy as np
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from multiprocessing import Pool
import os


# Function to load and preprocess a single image
def process_image(image_name):
    image_path = os.path.join(image_folder_path, image_name)
    image = cv2.imread(image_path)
    if image is None:
        print(f"Unable to open {image_path}. Skipping.")
        return None
    image = cv2.resize(image, (16, 16))  # use a lower resolution
    return image.flatten()

# Load dataframe
#df = pd.read_csv('your_dataframe.csv')

# Randomly sample 10,000 images from the dataframe
df_sample = df_hair.sample(n=25000, random_state=1)

# Load and preprocess images
image_folder_path = 'img_align_celeba/'

# Use multiprocessing to load and preprocess images
with Pool(os.cpu_count()) as p:
    images = p.map(process_image, df_sample['image_id'])

# Filter out any images that couldn't be opened
images = [image for image in images if image is not None]

# Convert list of images to numpy array
images = np.array(images)

# Prepare labels
labels = df_sample[['Black_Hair', 'Blond_Hair', 'Brown_Hair', 'Gray_Hair']].values
labels = np.argmax(labels, axis=1)  # convert from one-hot encoding to single label

# Split dataset into training set and test set
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

# Scale data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Train SVM
clf = svm.SVC(gamma='scale')
clf.fit(X_train, y_train)

# Check accuracy on test set
accuracy = clf.score(X_test, y_test)
print('Accuracy: ', accuracy)


Accuracy:  0.7408


Test with 50,000 samples: 

In [16]:
import cv2
import pandas as pd
import numpy as np
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from multiprocessing import Pool
import os


# Function to load and preprocess a single image
def process_image(image_name):
    image_path = os.path.join(image_folder_path, image_name)
    image = cv2.imread(image_path)
    if image is None:
        print(f"Unable to open {image_path}. Skipping.")
        return None
    image = cv2.resize(image, (16, 16))  # use a lower resolution
    return image.flatten()

# Load dataframe
#df = pd.read_csv('your_dataframe.csv')

# Randomly sample 10,000 images from the dataframe
df_sample = df_hair.sample(n=50000, random_state=1)

# Load and preprocess images
image_folder_path = 'img_align_celeba/'

# Use multiprocessing to load and preprocess images
with Pool(os.cpu_count()) as p:
    images = p.map(process_image, df_sample['image_id'])

# Filter out any images that couldn't be opened
images = [image for image in images if image is not None]

# Convert list of images to numpy array
images = np.array(images)

# Prepare labels
labels = df_sample[['Black_Hair', 'Blond_Hair', 'Brown_Hair', 'Gray_Hair']].values
labels = np.argmax(labels, axis=1)  # convert from one-hot encoding to single label

# Split dataset into training set and test set
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

# Scale data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Train SVM
clf = svm.SVC(gamma='scale')
clf.fit(X_train, y_train)

# Check accuracy on test set
accuracy = clf.score(X_test, y_test)
print('Accuracy: ', accuracy)


Accuracy:  0.744


In [5]:
hair_cols = ['image_id','Attractive', 'Bangs', 'Black_Hair', 'Blond_Hair', 'Brown_Hair', 'Gray_Hair', 'Male', 'Oval_Face', 'Receding_Hairline', 'Smiling']
df_hair = df[hair_cols]
df_hair.head()

,image_id,Attractive,Bangs,Black_Hair,Blond_Hair,Brown_Hair,Gray_Hair,Male,Oval_Face,Receding_Hairline,Smiling
0,000001.jpg,1,-1,-1,-1,1,-1,-1,-1,-1,1
1,000002.jpg,-1,-1,-1,-1,1,-1,-1,-1,-1,1
2,000003.jpg,-1,-1,-1,-1,-1,-1,1,-1,-1,-1
3,000004.jpg,1,-1,-1,-1,-1,-1,-1,-1,-1,-1
4,000005.jpg,1,-1,-1,-1,-1,-1,-1,-1,-1,-1


In [9]:
import cv2
import pandas as pd
import numpy as np
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from multiprocessing import Pool
import os


# Function to load and preprocess a single image
def process_image(image_name):
    image_path = os.path.join(image_folder_path, image_name)
    image = cv2.imread(image_path)
    if image is None:
        print(f"Unable to open {image_path}. Skipping.")
        return None
    image = cv2.resize(image, (16, 16))  # use a lower resolution
    return image.flatten()

# Load dataframe
#df = pd.read_csv('your_dataframe.csv')

# Randomly sample 10,000 images from the dataframe
df_sample = df_hair.sample(n=20000, random_state=1)

# Load and preprocess images
image_folder_path = 'img_align_celeba/'

# Use multiprocessing to load and preprocess images
with Pool(os.cpu_count()) as p:
    images = p.map(process_image, df_sample['image_id'])

# Filter out any images that couldn't be opened
images = [image for image in images if image is not None]

# Convert list of images to numpy array
images = np.array(images)

# Prepare labels
labels = df_sample[['Attractive', 'Bangs', 'Black_Hair', 'Blond_Hair', 'Brown_Hair', 'Gray_Hair', 'Male', 'Oval_Face', 'Receding_Hairline', 'Smiling']].values
labels = np.argmax(labels, axis=1)  # convert from one-hot encoding to single label

# Split dataset into training set and test set
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

# Scale data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Train SVM
clf = svm.SVC(gamma='scale')
clf.fit(X_train, y_train)

# Check accuracy on test set
accuracy = clf.score(X_test, y_test)
print('Accuracy: ', accuracy)


Accuracy:  0.62075


In [10]:
from sklearn.metrics import precision_recall_fscore_support

# Predict labels for test set
y_pred = clf.predict(X_test)

# Compute precision, recall, and F1-score
precision, recall, f1, _ = precision_recall_fscore_support(y_test, y_pred, average='weighted')
print('Precision:', precision)
print('Recall:', recall)
print('F1-score:', f1)

Precision: 0.5008475587853919
Recall: 0.62075
F1-score: 0.5345078615008201


/opt/anaconda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [11]:
from sklearn.metrics import confusion_matrix

# Compute confusion matrix
cm = confusion_matrix(y_test, y_pred)
print('Confusion Matrix:')
print(cm)

Confusion Matrix:
[[2120    2   25    0    0    8   82    0    0    0]
 [ 203    0    2    0    0    2   16    0    0    0]
 [ 265    0   62    0    0    0   67    0    0    0]
 [ 115    0    0    0    0    6   14    0    0    0]
 [ 169    0    5    0    0    0   22    0    0    0]
 [  40    0    0    0    0   49   59    0    0    0]
 [ 254    0   26    0    0   16  252    0    0    0]
 [  38    0    0    0    0    1   12    0    0    0]
 [  13    0    1    0    0    0    1    0    0    0]
 [  48    0    0    0    0    0    5    0    0    0]]


In [12]:
from sklearn.metrics import classification_report

# Generate classification report
report = classification_report(y_test, y_pred)
print('Classification Report:')
print(report)

Classification Report:
              precision    recall  f1-score   support

           0       0.65      0.95      0.77      2237
           1       0.00      0.00      0.00       223
           2       0.51      0.16      0.24       394
           3       0.00      0.00      0.00       135
           4       0.00      0.00      0.00       196
           5       0.60      0.33      0.43       148
           6       0.48      0.46      0.47       548
           7       0.00      0.00      0.00        51
           8       0.00      0.00      0.00        15
           9       0.00      0.00      0.00        53

    accuracy                           0.62      4000
   macro avg       0.22      0.19      0.19      4000
weighted avg       0.50      0.62      0.53      4000



/opt/anaconda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
import cv2
import pandas as pd
import numpy as np
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from multiprocessing import Pool
import os


# Function to load and preprocess a single image
def process_image(image_name):
    image_path = os.path.join(image_folder_path, image_name)
    image = cv2.imread(image_path)
    if image is None:
        print(f"Unable to open {image_path}. Skipping.")
        return None
    image = cv2.resize(image, (16, 16))  # use a lower resolution
    return image.flatten()

# Load dataframe
#df = pd.read_csv('your_dataframe.csv')

# Randomly sample 10,000 images from the dataframe
df_sample = df_hair.sample(n=50000, random_state=1)

# Load and preprocess images
image_folder_path = 'img_align_celeba/'

# Use multiprocessing to load and preprocess images
with Pool(os.cpu_count()) as p:
    images = p.map(process_image, df_sample['image_id'])

# Filter out any images that couldn't be opened
images = [image for image in images if image is not None]

# Convert list of images to numpy array
images = np.array(images)

# Prepare labels
labels = df_sample[['Attractive', 'Bangs', 'Black_Hair', 'Blond_Hair', 'Brown_Hair', 'Gray_Hair', 'Male', 'Oval_Face', 'Receding_Hairline', 'Smiling']].values
labels = np.argmax(labels, axis=1)  # convert from one-hot encoding to single label

# Split dataset into training set and test set
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

# Scale data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Train SVM
clf = svm.SVC(gamma='scale')
clf.fit(X_train, y_train)

# Check accuracy on test set
accuracy = clf.score(X_test, y_test)
print('Accuracy: ', accuracy)
